# Library

In [120]:
import pandas as pd
import numpy as np
import os
import re
import collections
import unidecode
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation
from functools import reduce
import seaborn as sns
from abbreviations import schwartz_hearst
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import ast
import math

[nltk_data] Downloading package wordnet to /Users/egarcia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [121]:
pd.set_option('display.max_colwidth', 100)

In [122]:
%matplotlib inline
from matplotlib import pyplot as plt

# Functions

In [123]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [124]:
nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/egarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
def clean_text(string):
    """
    A method to clean text 
    """
    
    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuation:
            if x != '/':
                string = string.replace(x, "")
            else:
                string = string.replace(x, " ")
    
    string = unidecode.unidecode(string)

#     # Converting the text to lower
#     string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in swords])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string 

In [126]:
def distance_levenshtein(str1, str2):
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

In [127]:
def normalize_lf(row):
    leven2 = []
    for i in row:
        for j in row:
            if i != j:
                long = max(len(i),len(j))
                ratio = distance_levenshtein(i,j)/long
                if ratio < 0.2:
                    leven2.append(j)
    if leven2:
        leven2 = set(leven2)
        lista = []
        for i in leven2:
            val = frec[frec['index'] == i]['long_form'].iloc[0]
            lista.append((i, val))
        lista = set(lista)
        most_freq = sorted(set(lista), key=lambda x: x[1], reverse = True)[0][0]
        sust = {}
        for i in set(leven2):
            sust[i] = most_freq
        
        return sust
    else:
        pass
    

In [128]:
def get_label(row):
    if row['long_form_x'] == row['long_form_y']:
        return 1
    else:
        return 0

In [129]:
def offsetA(row):
    return row['texto'].find(row['Mention_A'])
    
def offsetB(row):
    return row['texto'].find(row['Mention_B'])

def offsetB_end(row):
    return row['texto'].find(row['Mention_B']) + len(row['Mention_B'])

def offsetA_end(row):
    return row['Mention_A_StartOffset'] + len(row['Mention_A'])

In [164]:
def offset(row):
    return row['texto'].find(row['abrev'])

def offsetend(row):
    return row['StartOffset']+len(row['abrev'])

In [134]:
def defin_dictionary(row,dictionary):
    if row['Definition'] == 'no_existe':
        return dictionary.get(row['Abbreviation'])
    else:
        return row['Definition']

# Load Data

### Testing

220 clinical cases.

In [135]:
testing_abbr = pd.read_csv("../../acronym_disambiguation_tfm/data/ibereval_data/testing_set/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')

In [136]:
#testing_abbr = testing_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [137]:
testing_abbr.head()

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0212-71992005000500009-1,533,537,/mm3,milímetro cúbico,milímetro cúbico
1,S1130-14732005000200003-1,1857,1858,µ,micro,micro
2,S0211-69952013000200018-1,2512,2514,µg,microgramo,microgramo
3,S0212-16112004000400007-1,4558,4560,µg,microgramo,microgramo
4,S1137-66272014000300016-1,1112,1114,µg,microgramo,microgramo


In [138]:
testing_raw = read_texts("../../acronym_disambiguation_tfm/data/ibereval_data/testing_set/testing_set.raw_text/")

In [139]:
testing_raw.head()

,nombre,texto
0,S1130-01082008001000010-1,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d..."
1,S0004-06142009000400011-1,Varón de 75 años con antecedentes de EPOC moderado sin otros antecedentes médicos de interés. En...
2,S0376-78922011000200004-1,Mujer de 44 años de edad con antecedentes médicos de insuficiencia renal crónica en tratamiento ...
3,S1137-66272014000300016-1,Mujer de 36 años sin antecedentes médicos o epidemiológicos de interés que es ingresada en la Un...
4,S1130-01082008000800019-1,"Recientemente, atendimos en nuestro hospital a un varón de 46 años, con antecedente de síndrome ..."


### Trainning

318 clinical cases

In [140]:
train_abbr = pd.read_csv("../../acronym_disambiguation_tfm/data/ibereval_data/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')

In [141]:
#train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [142]:
train_abbr.Definition.nunique()

908

In [143]:
train_abbr.head()

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0210-48062004000500008-1,1650,1652,ml,mililitro,mililitro
1,S0210-48062004000500008-1,708,709,l,litro,litro
2,S0210-48062004000500008-1,704,707,mEq,miliequivalente,miliequivalente
3,S0210-48062004000500008-1,677,681,pCO2,presión parcial de co2,presión parcial de co2
4,S0210-48062004000500008-1,2287,2290,HLA,human leucocyte antigen,human leucocyte antiger


In [144]:
train_abbr = train_abbr.dropna()

In [145]:
train_abbr.to_csv("../../datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t', index = False)

In [146]:
train_raw = read_texts("../../acronym_disambiguation_tfm/data/ibereval_data/trainning_set/training_set.raw_text/")

In [147]:
#train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [148]:
train_raw.head()

,nombre,texto
0,S1130-01082006000100014-1,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec..."
1,S1130-01082009000300015-1,"Varón de 70 años, fumador, con enfisema pulmonar y vitíligo al que en mayo de 2001 se realizó un..."
2,S0210-56912010000200009-1,Se trata de una mujer de 70 años con antecedentes de HTA y diagnosticada recientemente de neopla...
3,S1130-01082008000900014-1,Varón de 41 años diagnosticado de adenocarcinoma medianamente diferenciado implantado sobre esóf...
4,S0210-48062004000500008-1,Paciente de 29 años de edad que acude al Servicio de Urgencias de nuestro Hospital ante la prese...


## Test with AbreMES DB

### 1) Get the short-form from the text

In [149]:
nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))
swords = swords + ['I','II','III','VI','VII','VIII','IX', 'X', 'x']

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/egarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [150]:
patron1 = r'[A-Z]{2,8}' #Letras mayúsculas entre 2 y 8. Probar (\s|\()[A-Z]{2,8}
patron2 = r'\s[a-z]{1,2}\s' #Entre 2 y 3 letras minusculas entre espacios
patron3 = r'\b[aA-zZ]{1,4}\-[aA-zZ]{1,4}\b' #mayúsuclas o minúsculas entre guiones
patron4 = r'\b\w{2}\b\/' #medidas antes de un signo /
patron5 = r'\/\b\w{2}\b' #medidas después de un signo /
patron6 = r'[aA-zZ]{1,4}[A-Z]+[a-z]*[1-4]*'
patron7 = r'\b\w{1,3}\s[0-9]{1,3}\b' #letras y numeros separadas por espacio
patron8 = r'\b\w{1,3}[-]\w{1,3}\b' #tras y numeros separas por guión
patron9= r'\b[aA-zZ]{1,3}[0-9]{1,3}\b' #letras con numeros todo junto
#patron8 = r'\/[a-z]*[A-Z]*'
#patron6 = r'[1-9]\s*[aA-zZ]{1,4}\/\b[aA-zZ]{1,4}\b' #Palabras divididas por / solo cuando las palabras no exceden de 4 caracteres


# create a list with them
regexes = [ patron1, patron2, patron3, patron4, patron5, patron6, patron7, patron8, patron9]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s|%s|%s|%s|%s" % (patron1, patron2, patron3, patron4, patron5, patron6, patron7, patron8, patron9))

### Training

In [151]:
train_raw['texto_clean'] = train_raw['texto'].str.split().map(lambda x: ' '.join([w for w in x if w not in swords]))

In [152]:
train_raw['abrev'] = train_raw['texto_clean'].map(lambda x: generic_re.findall(x))

In [153]:
train_raw.head()

,nombre,texto,texto_clean,abrev
0,S1130-01082006000100014-1,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...","Se trata mujer 35 años, antecedentes familiares enfermedad Crohn antecedentes patológicos interé...","[FID, A 2, IMC, /60, /dl, 700 84, GOT, GPT, GT, /dl, mg/, mg/, mm3 90, A 5, LCR]"
1,S1130-01082009000300015-1,"Varón de 70 años, fumador, con enfisema pulmonar y vitíligo al que en mayo de 2001 se realizó un...","Varón 70 años, fumador, enfisema pulmonar vitíligo mayo 2001 realizó colonoscopia diarrea, dolor...","[ cm , cm , pg/, pg/, mg , TC, cm , cm , cm , pg/, mg , cm , cm ]"
2,S0210-56912010000200009-1,Se trata de una mujer de 70 años con antecedentes de HTA y diagnosticada recientemente de neopla...,"Se trata mujer 70 años antecedentes HTA diagnosticada recientemente neoplasia recto bajo, tratad...","[HTA, UCI, VM, VMNI, oC, /dl, /μl, VM, oC, TC, BAL, PCR, CMV, BAL, PCR, VHS, /kg, /8h]"
3,S1130-01082008000900014-1,Varón de 41 años diagnosticado de adenocarcinoma medianamente diferenciado implantado sobre esóf...,Varón 41 años diagnosticado adenocarcinoma medianamente diferenciado implantado esófago Barrett ...,"[ mm , CRE, mm , mm , EE, UU, mmHg, mmHg]"
4,S0210-48062004000500008-1,Paciente de 29 años de edad que acude al Servicio de Urgencias de nuestro Hospital ante la prese...,Paciente 29 años edad acude Servicio Urgencias Hospital presencia erección levemente dolorosa di...,"[ u , pCO2, mmHg, HCO, mEq, mg/, ml , mg , 20-30, ml , 414 109, HLA]"


In [154]:
train_track2 = train_raw.explode('abrev')

In [155]:
train_track2['abrev'] = train_track2['abrev'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')

/var/folders/1q/pw51_8397cv2pn5m63n8dwzr0000gn/T/ipykernel_64120/1487880884.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_track2['abrev'] = train_track2['abrev'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')


In [163]:
train_track2.head()

,nombre,texto,texto_clean,abrev,StartOffset
0,S1130-01082006000100014-1,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...","Se trata mujer 35 años, antecedentes familiares enfermedad Crohn antecedentes patológicos interé...",FID,192
0,S1130-01082006000100014-1,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...","Se trata mujer 35 años, antecedentes familiares enfermedad Crohn antecedentes patológicos interé...",A 2,-1
0,S1130-01082006000100014-1,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...","Se trata mujer 35 años, antecedentes familiares enfermedad Crohn antecedentes patológicos interé...",IMC,1144
0,S1130-01082006000100014-1,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...","Se trata mujer 35 años, antecedentes familiares enfermedad Crohn antecedentes patológicos interé...",60,1193
0,S1130-01082006000100014-1,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...","Se trata mujer 35 años, antecedentes familiares enfermedad Crohn antecedentes patológicos interé...",dl,1414


In [157]:
train_track2 = train_track2[~train_track2.abrev.isnull()]

In [165]:
train_track2['StartOffset'] = train_track2.apply(offset, axis = 1)
train_track2['EndOffset'] = train_track2.apply(offsetend, axis = 1)

In [167]:
train_track2 = train_track2[['nombre','StartOffset', 'EndOffset' ,'abrev','texto']]

In [168]:
train_track2.head()

,nombre,StartOffset,EndOffset,abrev,texto
0,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec..."
0,S1130-01082006000100014-1,-1,2,A 2,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec..."
0,S1130-01082006000100014-1,1144,1147,IMC,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec..."
0,S1130-01082006000100014-1,1193,1195,60,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec..."
0,S1130-01082006000100014-1,1414,1416,dl,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec..."


### 2) Give a long-form from AbreMES data base

In [169]:
abremes = pd.read_csv("../../publicacion/AbreMES-DB/DB/pairs.tsv", sep = '\t')

In [170]:
abremes.head()

,# Pair ID,Abbreviation ID,Definition ID,Frequency,Abbreviation,Definition,Appears on
0,1,3348,17876,31,DDD,diaria definida,"http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300004,http://scielo..."
1,2,11880,23106,11,HP-CHPR,Hospital Pediátrico del Centro Hospitalario Pereira Rossell,"http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300004,http://www.sc..."
2,3,1454,23213,1,EVN,de vida al nacer,http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300005
3,4,1112,23214,1,TDS,Total dermatoscopic score,http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300006
4,5,231,23215,1,AP,cases by pathologic anatomy,http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300006


In [171]:
abremes['Abbreviation'] = abremes['Abbreviation'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')

/var/folders/1q/pw51_8397cv2pn5m63n8dwzr0000gn/T/ipykernel_64120/1011565268.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  abremes['Abbreviation'] = abremes['Abbreviation'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')


#### Train

In [172]:
train_track2 = train_track2.rename(columns = {'abrev':'Abbreviation'})

In [173]:
train_track2['Abbreviation'] = train_track2['Abbreviation'].str.strip()

In [174]:
print(train_track2.shape)
print(train_track2.Abbreviation.nunique())
print(abremes.shape)
print(abremes.Abbreviation.nunique())

(3834, 5)
1076
(52551, 7)
20236


Add dictionary with measure units

In [175]:
with open("dictionary_measureunits.txt", "r") as data:
    dictionary = ast.literal_eval(data.read())

Assign definitions from AbreMES DB

In [176]:
train_def = train_track2.merge(abremes[['Abbreviation', 'Definition']], how = 'left', on = 'Abbreviation', indicator = True)

In [177]:
train_def.head(2)

,nombre,StartOffset,EndOffset,Abbreviation,texto,Definition,_merge
0,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",Federación Internacional de Diabetes,both
1,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",Fósforo Inorgánico Disuelto,both


In [178]:
train_def['Definition'] = train_def['Definition'].fillna('no_existe')

In [179]:
train_def['Definition'] = train_def.apply(lambda x: defin_dictionary(x, dictionary), axis = 1)

In [180]:
sf_notfind= train_def[train_def['Definition'].isna()]['Abbreviation'].unique().tolist()

In [181]:
len(sf_notfind)

597

In [182]:
train_def= train_def.dropna(subset = ['Definition'])

In [183]:
#En test real hay 600 y pico
print(train_def.Abbreviation.nunique())

479


In [184]:
train_def.head()

,nombre,StartOffset,EndOffset,Abbreviation,texto,Definition,_merge
0,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",Federación Internacional de Diabetes,both
1,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",Fósforo Inorgánico Disuelto,both
2,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",formación inicial docente,both
3,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",factor de independencia a la distracción,both
4,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",fosa iliaca derecha,both


In [185]:
print(train_def.shape)
print(train_def.Abbreviation.nunique())

(46869, 7)
479


Get lemmatized long forms

In [186]:
lemmatizer = WordNetLemmatizer()

In [187]:
train_def['Definition_lemmatized'] = train_def['Definition'].map(lambda x: lemmatizer.lemmatize(x))

In [188]:
train_def.head()

,nombre,StartOffset,EndOffset,Abbreviation,texto,Definition,_merge,Definition_lemmatized
0,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",Federación Internacional de Diabetes,both,Federación Internacional de Diabetes
1,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",Fósforo Inorgánico Disuelto,both,Fósforo Inorgánico Disuelto
2,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",formación inicial docente,both,formación inicial docente
3,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",factor de independencia a la distracción,both,factor de independencia a la distracción
4,S1130-01082006000100014-1,192,195,FID,"Se trata de una mujer de 35 años, con antecedentes familiares de enfermedad de Crohn y sin antec...",fosa iliaca derecha,both,fosa iliaca derecha


In [227]:
del train_def['_merge']

In [228]:
train_def.to_csv("../data/data_paper/train_subtrack2_parte1.csv", index = False)

### Test

In [190]:
testing_raw['texto_clean'] = testing_raw['texto'].str.split().map(lambda x: ' '.join([w for w in x if w not in swords]))

In [191]:
testing_raw['abrev'] = testing_raw['texto_clean'].map(lambda x: generic_re.findall(x))

In [192]:
testing_raw.head()

,nombre,texto,texto_clean,abrev
0,S1130-01082008001000010-1,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...","Varón 43 años originario Marruecos, ingresó servicio cuadro 4 días evolución ictericia mucocután...","[mg/, GOT, UI, /ml, GPT, UI, /ml, GGT, UI, /ml, FA, UI, /ml, LDH, UI, /ml, IgG, UI, /ml, 751-1, ..."
1,S0004-06142009000400011-1,Varón de 75 años con antecedentes de EPOC moderado sin otros antecedentes médicos de interés. En...,"Varón 75 años antecedentes EPOC moderado antecedentes médicos interés. Entre hábitos tóxicos, tr...","[EPOC, cm , 2-3, cm , X10, EMA, TAC, RMN, TAC]"
2,S0376-78922011000200004-1,Mujer de 44 años de edad con antecedentes médicos de insuficiencia renal crónica en tratamiento ...,Mujer 44 años edad antecedentes médicos insuficiencia renal crónica tratamiento hemodiálisis año...,"[IV, 25 25, cm , mg/, mg/, mg/, mg/, VAC, mmHg, VAC, mmHg, A 6, IV, 2 3, VAC]"
3,S1137-66272014000300016-1,Mujer de 36 años sin antecedentes médicos o epidemiológicos de interés que es ingresada en la Un...,Mujer 36 años antecedentes médicos epidemiológicos interés ingresada Unidad Cuidados Intensivos ...,"[TAC, oC, µg/, VSG, PCR, mg/, VN, mg/, ng/, VIH, VEB, CMV, VHS, 1 2, ANA, ANCA, mg/, TAC, RMN, 7..."
4,S1130-01082008000800019-1,"Recientemente, atendimos en nuestro hospital a un varón de 46 años, con antecedente de síndrome ...","Recientemente, atendimos hospital varón 46 años, antecedente síndrome Down retraso mental grave,...","[TC, 24-48]"


In [193]:
test_track2 = testing_raw.explode('abrev')

In [194]:
test_track2['abrev'] = test_track2['abrev'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')

/var/folders/1q/pw51_8397cv2pn5m63n8dwzr0000gn/T/ipykernel_64120/2963199427.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_track2['abrev'] = test_track2['abrev'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')


In [195]:
test_track2.head()

,nombre,texto,texto_clean,abrev
0,S1130-01082008001000010-1,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...","Varón 43 años originario Marruecos, ingresó servicio cuadro 4 días evolución ictericia mucocután...",mg
0,S1130-01082008001000010-1,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...","Varón 43 años originario Marruecos, ingresó servicio cuadro 4 días evolución ictericia mucocután...",GOT
0,S1130-01082008001000010-1,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...","Varón 43 años originario Marruecos, ingresó servicio cuadro 4 días evolución ictericia mucocután...",UI
0,S1130-01082008001000010-1,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...","Varón 43 años originario Marruecos, ingresó servicio cuadro 4 días evolución ictericia mucocután...",ml
0,S1130-01082008001000010-1,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...","Varón 43 años originario Marruecos, ingresó servicio cuadro 4 días evolución ictericia mucocután...",GPT


In [196]:
test_track2 = test_track2[~test_track2.abrev.isnull()]

In [197]:
test_track2['StartOffset'] = test_track2.apply(offset, axis = 1)

In [198]:
test_track2['StartOffset'] = test_track2.apply(offset, axis = 1)
test_track2['EndOffset'] = test_track2.apply(offsetend, axis = 1)

In [201]:
test_track2 = test_track2[['nombre','StartOffset', 'EndOffset' ,'abrev','texto']]

In [202]:
test_track2.head()

,nombre,StartOffset,EndOffset,abrev,texto
0,S1130-01082008001000010-1,372,374,mg,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d..."
0,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d..."
0,S1130-01082008001000010-1,387,389,UI,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d..."
0,S1130-01082008001000010-1,390,392,ml,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d..."
0,S1130-01082008001000010-1,394,397,GPT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d..."


### 2) Give a long-form from AbreMES data base

In [203]:
abremes = pd.read_csv("../../publicacion/AbreMES-DB/DB/pairs.tsv", sep = '\t')

In [204]:
abremes.head()

,# Pair ID,Abbreviation ID,Definition ID,Frequency,Abbreviation,Definition,Appears on
0,1,3348,17876,31,DDD,diaria definida,"http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300004,http://scielo..."
1,2,11880,23106,11,HP-CHPR,Hospital Pediátrico del Centro Hospitalario Pereira Rossell,"http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300004,http://www.sc..."
2,3,1454,23213,1,EVN,de vida al nacer,http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300005
3,4,1112,23214,1,TDS,Total dermatoscopic score,http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300006
4,5,231,23215,1,AP,cases by pathologic anatomy,http://www.scielo.edu.uy/scielo.php?script=sci_arttext&pid=S1688-03902003000300006


In [205]:
abremes['Abbreviation'] = abremes['Abbreviation'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')

/var/folders/1q/pw51_8397cv2pn5m63n8dwzr0000gn/T/ipykernel_64120/1011565268.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  abremes['Abbreviation'] = abremes['Abbreviation'].str.replace('[!"#$%&*+,-./:;<=>?@^_`{|}~]','')


In [206]:
test_track2 = test_track2.rename(columns = {'abrev':'Abbreviation'})

In [207]:
test_track2['Abbreviation'] = test_track2['Abbreviation'].str.strip()

In [208]:
print(test_track2.shape)
print(test_track2.Abbreviation.nunique())
print(abremes.shape)
print(abremes.Abbreviation.nunique())

(3058, 5)
908
(52551, 7)
20236


Add dictionary with measure units

In [209]:
with open("dictionary_measureunits.txt", "r") as data:
    dictionary = ast.literal_eval(data.read())

Assign definitions from AbreMES DB

In [210]:
test_def = test_track2.merge(abremes[['Abbreviation', 'Definition']], how = 'left', on = 'Abbreviation', indicator = True)

In [211]:
test_def.head(2)

,nombre,StartOffset,EndOffset,Abbreviation,texto,Definition,_merge
0,S1130-01082008001000010-1,372,374,mg,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",NaN,left_only
1,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutamic-oxaloacetic transaminase,both


In [212]:
test_def['Definition'] = test_def['Definition'].fillna('no_existe')

In [213]:
test_def['Definition'] = test_def.apply(lambda x: defin_dictionary(x, dictionary), axis = 1)

In [214]:
sf_notfind= test_def[test_def['Definition'].isna()]['Abbreviation'].unique().tolist()

In [215]:
len(sf_notfind)

503

In [216]:
test_def= test_def.dropna(subset = ['Definition'])

In [217]:
#En test real hay 600 y pico
print(test_def.Abbreviation.nunique())

405


In [218]:
test_def.head()

,nombre,StartOffset,EndOffset,Abbreviation,texto,Definition,_merge
0,S1130-01082008001000010-1,372,374,mg,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",miligramo,left_only
1,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutamic-oxaloacetic transaminase,both
2,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutámico-oxalacetica,both
3,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutamato oxaloacetil transaminasa,both
4,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutamato oxaloacetato transaminasa,both


In [219]:
print(test_def.shape)
print(test_def.Abbreviation.nunique())

(46331, 7)
405


Get lemmatized long forms

In [220]:
lemmatizer = WordNetLemmatizer()

In [221]:
test_def['Definition_lemmatized'] = test_def['Definition'].map(lambda x: lemmatizer.lemmatize(x))

In [222]:
test_def.head()

,nombre,StartOffset,EndOffset,Abbreviation,texto,Definition,_merge,Definition_lemmatized
0,S1130-01082008001000010-1,372,374,mg,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",miligramo,left_only,miligramo
1,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutamic-oxaloacetic transaminase,both,glutamic-oxaloacetic transaminase
2,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutámico-oxalacetica,both,glutámico-oxalacetica
3,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutamato oxaloacetil transaminasa,both,glutamato oxaloacetil transaminasa
4,S1130-01082008001000010-1,379,382,GOT,"Varón de 43 años originario de Marruecos, que ingresó en nuestro servicio por cuadro de 4 días d...",glutamato oxaloacetato transaminasa,both,glutamato oxaloacetato transaminasa


In [225]:
del test_def['_merge']

In [226]:
test_def.to_csv("../data/data_paper/test_subtrack2_parte1.csv", index = False)